# Portfolio 1

1号组合是一个被动组合，主要关注沪深300、中证500、科创50、北证50、MSCI中国A50指数。

因为还没有找到合适的券商交易API，为便于操作和计算，选取了对应的场外ETF联接基金，这样可以通过在支付宝中设置定投实现买入操作，分别是：

- 沪深300：易方达沪深300ETF联接C(007339)
- 中证500：易方达中证500ETF联接C(007029)
- 科创50：易方达上证科创50联接C(011609)
- 北证50：易方达北证50成份指数C(017516)
- MSCI中国A50：易方达MSCI中国A50互联互通ETF联接C(014533)

## 交易策略

为方便计算，假设本金10万元，5个指数均等分配，每个指数2万元。再将每个指数的2万元分成100份进行定投。

### 买入

1. 沪深300和MSCI中国A50每周二、四定投2次，采用涨跌幅策略。
2. 中证500、科创50和北证50每周二、三、四定投3次，采用涨跌幅策略。

### 卖出

每个交易日14:30检查指数成交价或基金估算净值：

1. 当投入份额小于25%时，持有收益大于6%，则全部卖出。
2. 当投入份额小于50%时，持有收益大于8%，则全部卖出。
3. 当投入份额小于75%时，持有收益大于10%，则全部卖出。
4. 当投入份额大于75%时，持有收益大于12%，则全部卖出。

## 数据源

1. [中证指数](https://www.csindex.com.cn/#/indices/family/detail?indexCode=000300)
2. [蛋卷基金](https://danjuanfunds.com/djmodule/value-center?channel=1300100141)
3. [雪球基金](https://danjuanfunds.com/funding/011609?channel=XQWEB0001)
4. [东方财富](https://quote.eastmoney.com/sh560050.html)
5. [同花顺](https://fund.10jqka.com.cn/011609/)

### 指数数据

https://www.csindex.com.cn/csindex-home/perf/index-perf-oneday?indexCode=000300

In [17]:
import httpx

# 请求URL
url = "https://www.csindex.com.cn/csindex-home/perf/index-perf-oneday?indexCode=000300"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
cookie_str = "Hm_lvt_f0fbc9a7e7e7f29a55a0c13718e9e542=1707731968; aliyungf_tc=8713c50f0368db433c31293d97c851c2c44c7a377f5d75651aad6a28a0babf77; acw_tc=ac11000117081847345634446e8dcaf92c6e1283c6273190f327fddd141070; acw_sc__v3=65d0d4a5c34bfd7b7db503ddb4ffacbc25ddb908; acw_sc__v2=65d0d4dd3b3864eaf0de6f0c737e65d66bc143fd; tfstk=eh8vWBt5PYDc7TyFqqnk7JGWa_GoqEdVVKRQSOX0fLpJOBycfxWmWhpepRjs3ZY92d9lhE_XmhC9nd-0oZGNWFpDFSXjQj51WKvWBGlVIO69ICtmZ20HgI7N5PHn-2YZpoWjKKuO4_AV7NMoYCyzhIJ3fyoPxXM3-5NYzU-AGDia8hAtNU1AD9UGePUtssIAdI6-dvT-QiBBGTU14R8HJpiLK9Cgcbh8blSfaQjVFstQdFQVw9cfclrNj_zgDFyQblSra_BnGPZabGfP.; Hm_lpvt_f0fbc9a7e7e7f29a55a0c13718e9e542=1708184801; ssxmod_itna=YqAxyD9iDtIqgDl+D+ob=D7FncgrlGQexQTEPPDsWDcexA5D8D6DQeGTrnQOrqOKn8wxaqdZLP+QjYiGEi3q7=B2W8pYn+pasFDCPGnDBFobDwDYYCDt4DTD34DYDix5DLDmeD+RvKDdRh5s/o5Drxi1DeKDmxGCDlICDjnbOmr1ODDzHzoPOWb8Dm+CMPIwDWKDwxD1pj5PWY5D9x0CDlady+oD0hLUOeBzXKLAf0Kmi40OD0KfRpy7f4Gdc=7sw/qxfWie/Q04rjPPAjDYGf0PNBrj6e4xHYP5kWRKHBZZODDWs8hH4D==; ssxmod_itna2=YqAxyD9iDtIqgDl+D+ob=D7FncgrlGQexQToPikEWpDcirQD0yDcq03QUB6KdOC=qidqFGdqUPedpQgRhpKGWoTzo=0lBT6WHzDyiqp0aRDpQWIOGyiQpHToj=CNI=8tgpUYHmkp0Uxvjzo0wF8De4mvpzu=gjT2hWQLm3fUOjWUpnCiIA3IQ/QoeYbUdLQQZIgWPydQHXq9fQ79pROAqOcRC8WTU0P6f4Idh1LBs5tjCjqyCRROQpj9X4TH4I=LEK4kM9I284qk6qqCdHZnEBCUfjWZQOsZA8HczjMCCAxsZw5/gp51cutEuTUL3kk3qjsKQdv8TjfseerZqh78kLfNNmGmMbTn=axkVkKZg1hQhwm12DpHbYInTZhQ4ow7Rpe++xSGZkIzlmRxdhMwRMhkTwBeq/Tt=ndY65pSwxcWGRE/j5lofVlbCgwNoPD7QrxGcDG7OiDD"

cookies = {item.split("=")[0]: item.split("=")[1] for item in cookie_str.split("; ")}
response = httpx.get(url, headers=headers, cookies=cookies, verify=False)

# 打印响应内容
print(response.content.decode("utf-8"))

{"code":"200","msg":"Success","data":{"intraDayHeader":{"indexCode":"000300","tradeDate":"2024-02-08","tradeTime":"16:29:56","openToday":3351.41,"closePre":3343.63,"current":3364.93,"change":21.3,"changePct":0.64,"tradingVol":18997.76,"tradingValue":2702.4},"intraDayPerfList":[{"indexCode":"000300","indexName":"沪深300","indexNameEn":"CSI 300","tradeDate":"2024-02-08","tradeTime":"09:30:10","current":3351.63,"high":3351.91,"low":3351.41,"close":0.0,"change":8.0,"changePct":0.24},{"indexCode":"000300","indexName":"沪深300","indexNameEn":"CSI 300","tradeDate":"2024-02-08","tradeTime":"09:30:25","current":3347.09,"high":3351.91,"low":3347.09,"close":0.0,"change":3.46,"changePct":0.1},{"indexCode":"000300","indexName":"沪深300","indexNameEn":"CSI 300","tradeDate":"2024-02-08","tradeTime":"09:30:40","current":3346.41,"high":3351.91,"low":3344.85,"close":0.0,"change":2.78,"changePct":0.08},{"indexCode":"000300","indexName":"沪深300","indexNameEn":"CSI 300","tradeDate":"2024-02-08","tradeTime":"09:30

### 基金净值

1. https://fund.eastmoney.com/007029.html?spm=search
2. https://xueqiu.com/S/SH560050
3. https://danjuanfunds.com/djapi/fund/nav/history/011609?page=1&size=20
4. https://fundgz.1234567.com.cn/js/011609.js

In [22]:
import httpx

# 请求URL
url = "https://fund.10jqka.com.cn/data/client/myfund/011609"
response = httpx.get(url)

# 打印响应内容
print(response.json())

jsonpgz({"fundcode":"011609","name":"易方达上证科创50联接C","jzrq":"2024-02-07","dwjz":"0.5902","gsz":"0.5963","gszzl":"1.04","gztime":"2024-02-08 15:00"});
